# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

Question 1: 8.382332347441762
Question 2: 3.605551275463989

In [26]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)    

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [27]:
question_1 = round(sum([val for val in square_root_generator(5)]), 3)
question_1

8.382

In [32]:
limit = 13
question_2 = round([val for val in square_root_generator(limit)][limit-1], 3)
question_2

3.606

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [3]:
import dlt
import duckdb

In [34]:
generators_pipeline = dlt.pipeline(
    destination='duckdb', 
    dataset_name='people'
)

result_output = generators_pipeline.run(
    people_1(), 
    table_name='my_people', 
    write_disposition='replace'
)

result_output = generators_pipeline.run(
    people_2(), 
    table_name='my_people', 
    write_disposition='append'
)

In [46]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
my_df = conn.sql(f"""SELECT * FROM people.my_people""").df()

display(my_df)
question_3 = conn.sql('SELECT sum(age) FROM people.my_people').fetchone()[0]
print(f'Answer for Question 3: {question_3}')

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708163734.233005,EJsOvqRL1s5fZw,None
1,2,Person_2,27,City_A,1708163734.233005,YSBEISB2hGtVgQ,None
2,3,Person_3,28,City_A,1708163734.233005,hRO2s0P1IY8HDQ,None
3,4,Person_4,29,City_A,1708163734.233005,/F2YnD7Drz1mzQ,None
4,5,Person_5,30,City_A,1708163734.233005,61TDWAWeYwn4xg,None
5,3,Person_3,33,City_B,1708163734.8509338,a2cmnS/Tg24tTw,Job_3
6,4,Person_4,34,City_B,1708163734.8509338,cDMrKAMFpftj9w,Job_4
7,5,Person_5,35,City_B,1708163734.8509338,XUgZqE3zX+h2xQ,Job_5
8,6,Person_6,36,City_B,1708163734.8509338,9wRzZ2cWbuH2NQ,Job_6
9,7,Person_7,37,City_B,1708163734.8509338,USNddo00OIuj0w,Job_7


Answer for Question 3: 353


Question 3: 353

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [41]:
generators_pipeline = dlt.pipeline(
    destination='duckdb', 
    dataset_name='people')

result_output = generators_pipeline.run(
    people_1(), 
    table_name='new_people', 
    write_disposition='replace',
    primary_key='id'
)

result_output = generators_pipeline.run(
    people_2(), 
    table_name='new_people', 
    write_disposition='merge',
    primary_key='id'
)

In [47]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
my_df = conn.sql(f"""SELECT * FROM people.new_people""").df()

display(my_df)
question_4 = conn.sql('SELECT sum(age) FROM people.new_people').fetchone()[0]
print(f'Answer for Question 4: {question_4}')

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708163875.7542107,4jbF21dUEHjxIQ,None
1,2,Person_2,27,City_A,1708163875.7542107,1qMt56UYQuS8HA,None
2,5,Person_5,35,City_B,1708163876.3488834,aLwYz8PPRnptdw,Job_5
3,3,Person_3,33,City_B,1708163876.3488834,wx8qLsvlk5dXZA,Job_3
4,6,Person_6,36,City_B,1708163876.3488834,hUYNd7xUVlBf9w,Job_6
5,8,Person_8,38,City_B,1708163876.3488834,C2dEmHJ2aP+acw,Job_8
6,4,Person_4,34,City_B,1708163876.3488834,GO96RBx+IhcJNA,Job_4
7,7,Person_7,37,City_B,1708163876.3488834,B/XITLQPc37rrA,Job_7


Answer for Question 4: 266


Question 4: 266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX